In [ ]:
import os

import geopandas as gpd
import gspread
import pandas as pd

from dotenv import load_dotenv
from sqlalchemy import create_engine
from unidecode import unidecode

load_dotenv()

In [ ]:
# Geometry helpers to test simplification
def get_num_coords(poly):
    return len(poly.exterior.coords)

def get_total_num_coords(geom):
    if geom.geom_type == 'MultiPolygon':
        return sum(get_num_coords(poly) for poly in geom)
    elif geom.geom_type == 'Polygon':
        return get_num_coords(geom)
    else:
        raise ValueError('Geometry type must be either "Polygon" or "MultiPolygon"')

def get_pg_engine():
    connection_string = 'postgresql://{user}:{password}@{host}:{port}/{db}'.format(
        user=os.environ['PG_USER'],
        password=os.environ['PG_PASSWORD'],
        host=os.environ['PG_HOST'],
        port=os.environ['PG_PORT'],
        db=os.environ['PG_DATABASE']
    )
    
    return create_engine(connection_string)


def format_column(column_name):
    """Format a column name to be a valid BigQuery identifier"""
    chars_to_remove = "¿?(),."
    chars_to_replace = {"/": "_", " ": "_", "%": "pct"}
    for char in chars_to_remove:
        column_name = column_name.replace(char, "")
    for original_char, new_char in chars_to_replace.items():
        column_name = column_name.replace(original_char, new_char)
        
    # Multiple underscores into a single one
    column_name = '_'.join([part for part in column_name.split('_') if part != ''])

    column_name = unidecode(column_name.lower())
    return column_name


def gdf_from_df(df):
    """Construct a GeoDataFrame from a Pandas' DataFrame
    
    Assume the original DataFrame contains point information
    in fields `latitude` and `longitude`"""
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['longitude'], df['latitude']))
    gdf = gdf.drop(columns=['latitude', 'longitude'])
    gdf = gdf.set_crs(epsg=4326)
    
    return gdf


def fetch_camps_data():
    gspread_client = gspread.service_account('../campateca/campateca_sheets_service_account.json')

    sheet = gspread_client.open_by_key(os.environ['CAMPATECA_SHEET_ID']).worksheet('campateca')
    df = pd.DataFrame(sheet.get_all_records())
    df = df.replace({'': None, '#N/A': None})
    df.columns = df.columns.map(format_column)

    cols_to_drop = ['base_de_datos', 'como_contactar_quien_y_como', 'info_pleyades_hemos_llamado_visitado_cuando']
    cols_to_rename = {
        'si_has_estado_valora_el_campa_del_1_al_5': 'valoracion',
        'con_cuantas_plazas': 'num_plazas_cabanas_habitaciones',
        'breve_descripcion_info_adicional_ej_mola_para_rutas': 'comentarios',
    }
    boolean_cols = ['agua_corriente', 'electricidad', 'aseos', 'cocina', 'comedor', 'cabanas_habitaciones']

    df = df.drop(columns=cols_to_drop)
    df = df.rename(columns=cols_to_rename)
    
    # Drop empty rows
    df = df.loc[df['municipio'].notnull()]
    
    return df


def upload_table_create_view(gdf, table_name):
    view_name = f'{table_name}_geojson'
    conn.execute(f'DROP VIEW IF EXISTS {view_name}')
    gdf.to_postgis(table_name, conn, if_exists='replace')
    view_q = """
    CREATE OR REPLACE VIEW {table_name}_geojson AS (
      SELECT
        json_build_object(
          'type', 'FeatureCollection',
          'features', json_agg(ST_AsGeoJSON(c.*)::json)
        ) AS geojson
      FROM {table_name} c
    )"""
    
    conn.execute(view_q.format(table_name=table_name))


pg_engine = get_pg_engine()
conn = pg_engine.connect()

In [ ]:
df = fetch_camps_data()
gdf = gdf_from_df(df)

Write file locally as GeoJSON

In [ ]:
with open('campas.geojson', 'w') as fp:
    fp.write(gdf.loc[:40].to_json())

Upload to database

In [ ]:
upload_table_create_view(gdf, 'campamentos')

## Cuencas hidrográficas

In [ ]:
cuencas_cols = ['localId', 'nameText', 'rbdName', 'rbdArea', 'rbdAreaExc', 'internatio', 'interName', 'versionId', 'geometry']
cuencas_rename = {
    'localId': 'id',
    'nameText': 'name_esp',
    'rbdName': 'name_eng',
    'rbdArea': 'area',
    'rbdAreaExc': 'area_exc',
    'internatio': 'international',
    'interName': 'name_international',
    'versionId': 'version',
}

cuencas = gpd.read_file('data/demarcaciones_hidograficas/')[cuencas_cols]
cuencas = cuencas.rename(columns=cuencas_rename)

cuencas = cuencas.to_crs(epsg=4326)

cuencas.geometry = cuencas.geometry.simplify(0.001) # Good simplification without looking ugly

upload_table_create_view(cuencas, 'cuencas')

## Comunidades Autónomas

In [ ]:
shapefiles = {
    'peninbal': 'data/SHP_ETRS89/recintos_autonomicas_inspire_peninbal_etrs89/',
    'canarias': 'data/SHP_WGS84/recintos_autonomicas_inspire_canarias_wgs84/'
}

gdfs = []
for fp in shapefiles.values():
    gdf = gpd.read_file(fp)
    gdf = gdf.to_crs(epsg=4326)
    
    gdfs.append(gdf)

gdf = pd.concat(gdfs, ignore_index=True)
gdf.geometry = gdf.geometry.simplify(0.001)

upload_table_create_view(gdf, 'ccaa')